In [8]:
!pip install paho-mqtt

In [9]:
import pickle

model = pickle.load(open("esp8266_classifier.pkl", "rb"))

In [12]:
import paho.mqtt.client as mqtt

import json

import numpy as np

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("RM1/a85cb5/pub")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    
    txt = str(msg.payload)
    
    if (txt.find("&fields=monitor,MQTT_ID,time,millis,max_loop_wdt_cnt,FreeHeap,minFreeHeap,fs_usedBytes,wifi_rssi ") >= 0):
        txt = txt[txt.find("{"):len(txt)-1]

        json_data = json.loads(txt)
        print(json_data)

        element = [json_data['FreeHeap'],json_data['wifi_rssi'],json_data['fs_usedBytes'],json_data['max_loop_wdt_cnt']]
        print(model.predict(np.array(element).reshape(1, -1)))
        print("")

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker_address", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

Connected with result code 0
{'time': 1626627009, 'fs_usedBytes': 340858, 'millis': 632627051, 'max_loop_wdt_cnt': 992, 'MQTT_ID': 'RM1/a85cb5', 'FreeHeap': 39112, 'minFreeHeap': 32176, 'wifi_rssi': -75}
[1]



KeyboardInterrupt: 